In [34]:
from dotenv import load_dotenv
load_dotenv()

True

In [35]:
# !pip install playwright > /dev/null
# !pip install  lxml

# If this is your first time using playwright, you'll have to install a browser executable.
# Running `playwright install` by default installs a chromium browser executable.
# playwright install

In [23]:
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import run_async

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

In [48]:
from playwright.async_api import async_playwright
from news_breaker.get_elements import GetElementsToolUtf8

playwright = async_playwright()
device = {
    "user_agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.53 Safari/537.36",
    "screen": {
      "width": 1920,
      "height": 1080
    },
    "viewport": {
      "width": 1280,
      "height": 720
    },
    "device_scale_factor": 1,
    "is_mobile": False,
    "has_touch": False,
    "default_browser_type": "chromium"
}
browser = run_async(playwright.start())
browser = run_async(browser.chromium.launch(headless=True))
context = await browser.new_context(**device)

toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=browser)
tools = toolkit.get_tools()

tools_by_name = {tool.name: tool for tool in tools}
navigate_tool = tools_by_name["navigate_browser"]
get_elements_tool = GetElementsToolUtf8.from_browser(async_browser=browser)
extract_text_tool = tools_by_name["extract_text"]

url = "https://www.ftvnews.com.tw/news/detail/2023505W0297"
await navigate_tool.arun({"url": url})

'Navigating to https://www.ftvnews.com.tw/news/detail/2023505W0297 returned status code 200'

In [49]:
await get_elements_tool.arun({"selector": "article"})

'[{"innerText": "趙怡翔說：國民黨頭痛的是，該如何把郭台銘「輕輕地放下」。\\n\\n論壇中心／林靜芬報導\\n\\n國民黨2024總統人選尚未底定，新北市長侯友宜與鴻海創辦人郭台銘實力相當，但如今傳出國民黨已內定侯出馬參選。對此，民進黨台北市議員趙怡翔在《全國第一勇》節目中表示，郭台銘現在已經亂了、急了，「因為他知道他快被做掉了！」而國民黨接下來要思考的是，「如何把郭台銘輕輕地放下，一旦放得太快、破碎了，對不起，他又再次跟你國民黨翻臉」。\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n更多新聞： 快新聞／出獄畫面曝！\u3000趙玉柱獲假釋手比讚：非常高興\\n\\n郭台銘與侯友宜的國民黨總統參選徵召之爭，越趨白熱化。《全國第一勇》來賓趙怡翔指出，郭台銘最近拋出的許多議題，被批評欠缺周全的思考，包括「我要用AI去處理詐騙、我要用機器人去維護台灣的戰場」等等，為什麼這樣說？「因為他急了，他知道在整個過程當中已經都被內定了」。\\n\\n\\n\\n\\n\\n\\n\\n趙怡翔：侯、郭像打電動兩兄弟， 只有一個人真玩。（圖／民視新聞）\\n\\n\\n\\n\\n趙怡翔分析指出，郭台銘最近的處境，讓他想到網路上一張迷因哏圖，「就是兩個兄弟在打電動，兩個都打得很認真，但只有哥（侯友宜）遙控器有插進電玩裡面，弟弟（郭台銘）的是完全沒有插進去。弟弟就是在打假球，自high而已」。\\n\\n趙怡翔進一步表示：「所以我就說，只有侯友宜真的在玩，郭台銘以為他在玩，但他連插頭都沒插進去，不知道在玩什麼，但重點是，根本從頭到尾就沒有他的份，只不過還是有畫面」、「因為侯友宜也在按，就是哥哥也在按，弟弟按得很開心，以為是他在跳，但其實是哥哥在跳。」\\n\\n但郭董可以任憑國民黨愚弄嗎？趙怡翔認為，現在國民黨如果要說服大家，這個是一個公正的選舉，就應該把民調的基準、時間點拿出來，「你把到時候考慮的，不同因素跟百分比全部都拿出來，說對不起，客觀而言就是侯友宜比較強，我覺得這會買單的」，但是今天如果你是黑箱作業，說不出來任何的依據，最後就推侯友宜的話，國民黨會有麻煩。\\n「為什麼？你把他放得太快、 破碎了，他又再次跟你國民黨翻臉，甚至於去加入第三黨，所以國民黨現在要思考的就是，要如何把郭台銘輕輕地放下。」\\n\\n更多新聞： 賴清德新北民調超車侯友宜

In [41]:
from langchain.tools.python.tool import PythonREPLTool

python_repl_tool = PythonREPLTool()

In [44]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models.openai import ChatOpenAI
from langchain.callbacks import get_openai_callback

# llm = ChatOpenAI(model_name="gpt-4-0314", temperature=0)
llm = ChatOpenAI(temperature=0)

agent_chain = initialize_agent([navigate_tool, get_elements_tool], llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

prompt = f"""
1. 根據連結使用 selector: article, attributes: [innerText] 取得 innerText
2. 以台灣慣用詞彙產生100字以內的繁體中文總結。

連結：{url}
"""

with get_openai_callback() as cb:
    result = await agent_chain.arun(prompt)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://www.ftvnews.com.tw/news/detail/2023505W0297"
  }
}
```


Observation: Navigating to https://www.ftvnews.com.tw/news/detail/2023505W0297 returned status code 200
Thought:Action:
```
{
  "action": "get_elements",
  "action_input": {
    "selector": "article",
    "attributes": ["innerText"]
  }
}
```

Observation: [{"innerText": "\u8d99\u6021\u7fd4\u8aaa\uff1a\u570b\u6c11\u9ee8\u982d\u75db\u7684\u662f\uff0c\u8a72\u5982\u4f55\u628a\u90ed\u53f0\u9298\u300c\u8f15\u8f15\u5730\u653e\u4e0b\u300d\u3002\n\n\u8ad6\u58c7\u4e2d\u5fc3\uff0f\u6797\u975c\u82ac\u5831\u5c0e\n\n\u570b\u6c11\u9ee82024\u7e3d\u7d71\u4eba\u9078\u5c1a\u672a\u5e95\u5b9a\uff0c\u65b0\u5317\u5e02\u9577\u4faf\u53cb\u5b9c\u8207\u9d3b\u6d77\u5275\u8fa6\u4eba\u90ed\u53f0\u9298\u5be6\u529b\u76f8\u7576\uff0c\u4f46\u5982\u4eca\u50b3\u51fa\u570b\u6c11\u9ee8\u5df2\u5167\u5b9a\u4faf\u51fa\u99ac\u53c3\u9078\u3002\u5c